In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids, and separate the dataset:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [3]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [5]:
# We Separate the dataset
from implementations import *

tX_list, ids_list, y_list = separate_dataset(tX, ids, y, True)
tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test) 

## Grid Search

In [33]:
function = ridge_regression
degree_vec = []
lambda_vec = []
for i in range(6):
    print('Dataset part {l}:'.format(l = i))
    rmse_te, BestDeg, BestLambda, _ = grid_search(y_list[i], tX_list[i], function, True, 4, degrees= range(1, 6), lambdas=np.logspace(-8,-1,20))
    degree_vec.append(BestDeg)
    lambda_vec.append(BestLambda)
    print('     Best degree: {d}'.format(d = BestDeg))
    print('     Best lambda: {m}'.format(m = BestLambda))
    print('     Loss: {lo}'.format(lo = rmse_te))


Dataset part 0:
Done Deg
     Best degree: 2
     Best lambda: 1.6237767391887209e-06
     Loss: 0.41665525025152783
Dataset part 1:
Done Deg
     Best degree: 1
     Best lambda: 1.274274985703132e-07
     Loss: 0.7728515573320198
Dataset part 2:
Done Deg
     Best degree: 2
     Best lambda: 0.0006158482110660254
     Loss: 0.5230560284718709
Dataset part 3:
Done Deg
     Best degree: 4
     Best lambda: 1e-08
     Loss: 0.8181596533837344
Dataset part 4:
Done Deg
     Best degree: 3
     Best lambda: 0.0014384498882876629
     Loss: 0.5456309193330676
Dataset part 5:
Done Deg
     Best degree: 5
     Best lambda: 1e-08
     Loss: 0.7575400712773024


In [34]:
weights_list = []
loss_list = []
mat_tX_test_list = []

for i in range(6):
    mat_tX, mat_tX_test = build_poly_log(tX_list[i], degree_vec[i], True, tX_test_list[i])
    w, l = ridge_regression(y_list[i], mat_tX, lambda_vec[i])
    weights_list.append(w)
    loss_list.append(l)
    mat_tX_test_list.append(mat_tX_test)
    

## Gradient Descent

In [ ]:
# Linear regression using gradient descent

initial_w = np.zeros(tX.shape[1])
max_iters = 50
gamma = 3e-7

weights, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)

## Least squares

In [76]:
# Least squares regression using normal equation
weights, loss = least_squares(y, tX)
loss

0.3396868094770349

## Ridge regression

In [9]:
# Ridge regression using normal equations
lambda_ = 1e-3
weights, loss = ridge_regression(y, tX, lambda_)
loss

(30,)

In [59]:
# Ridge regression with data augmentation
lambda_ = 1e-3
mat_tX = build_poly(tX, 5)
weights, loss = ridge_regression(y, mat_tX, lambda_)

loss

0.2967432202164457

In [35]:
# Ridge regression with separated data
lambda_ = 1e-3
weights_list, loss_list = separated_train(tX_list, y_list, ridge_regression, lambda_)

In [76]:
# Ridge regression on separated datset with data augmentation
lambda_ = 1e-3
mat_tX_list = build_poly_separated(tX_list, 3)
# weights, loss = ridge_regression(y_list[0], mat_tX_list[0], lambda_)

weights_list, loss_list = separated_train(mat_tX_list, y_list, ridge_regression, lambda_)

## Logistic regression

In [5]:
tX_list, ids_list, y_list = separate_dataset(tX, ids, y, True)
tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test) 

degrees = range(1, 5)
lambdas = np.logspace(-6, 0, 3)
gammas = np.logspace(-3, 0, 3)

function = reg_logistic_regression
log = True

degree_vec = []
lambda_vec = []
gamma_vec = []

for i in range(6):
    rmse_te, BestDeg, BestLambda, BestGamma = grid_search(y_list[i], tX_list[i], function, log, 4, degrees, lambdas, gammas)
    degree_vec.append(BestDeg)
    lambda_vec.append(BestLambda)
    gamma_vec.append(BestGamma)
    print(rmse_te)

print(degree_vec)
print(lambda_vec)
print(gamma_vec)


0.7517759678635998
2.3291376629231237
1.4477112935903007
4.533192790692854
3.788780652676787
8.585765475873572
[1, 1, 1, 1, 1, 1]
[1.0, 0.001, 0.001, 1.0, 0.001, 1e-06]
[0.001, 0.001, 0.001, 0.001, 0.001, 0.001]


In [7]:
weights_list = []
loss_list = []
mat_tX_test_list = []
max_iter= 10000

for i in range(6):
    print("loop")
    mat_tX, mat_tX_test = build_poly_log(tX_list[i], degree_vec[0], True, tX_test_list[i])
    initial_w = np.zeros((mat_tX.shape[1], 1))
    l, w = reg_logistic_regression(y_list[i], mat_tX, initial_w, max_iter, lambda_vec[i], gamma_vec[i])
    print(l)
    weights_list.append(w)
    loss_list.append(l)
    mat_tX_test_list.append(mat_tX_test)

loop
0.29496869932801223
loop
2.282556239390623
loop
1.184794850905824
loop
7.410371791922858
loop
7.291330772326153
loop
22.49985375634216


## Generate predictions and save ouput in csv format for submission:

In [ ]:
# Data augmentation
OUTPUT_PATH = 'result.csv' # TODO: fill in desired name of output file for submission
mat_tX_test = build_poly(tX_test, 3)
print(mat_tX_test.shape)
y_pred = predict_labels(weights, mat_tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [41]:
# Separated dataset
OUTPUT_PATH = 'result.csv' # TODO: fill in desired name of output file for submission

tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test)
y_pred_list = separated_eval(weights_list, tX_test_list)

y_pred = np.concatenate((y_pred_list[0], y_pred_list[1], y_pred_list[2], y_pred_list[3]))
ids_test_sub = np.concatenate((ids_test_list[0], ids_test_list[1], ids_test_list[2], ids_test_list[3]))

create_csv_submission(ids_test_sub, y_pred, OUTPUT_PATH)

In [12]:
# Data augmentation and separated dataset
OUTPUT_PATH = 'result.csv' # TODO: fill in desired name of output file for submission

tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test)   #separate dateset

mat_tX_test_list = []

# for i in range(4):
#     print(i)
#     mat_tX_test = build_poly(tX_test_list[i], degree_vec[i])
#     mat_tX_test_list.append(mat_tX_test)

# mat_tX_test_list = build_poly_separated(tX_test_list, 3)            #data augmentation
y_pred_list = separated_eval(weights_list, mat_tX_test_list)        #prediction

y_pred = np.concatenate((y_pred_list[0], y_pred_list[1], y_pred_list[2], y_pred_list[3]))
ids_test_sub = np.concatenate((ids_test_list[0], ids_test_list[1], ids_test_list[2], ids_test_list[3]))

create_csv_submission(ids_test_sub, y_pred, OUTPUT_PATH)

0
1
2
3


## Prediction

In [35]:
# Eval

y_pred_list = separated_eval(weights_list, mat_tX_test_list, True) 

y_pred = np.concatenate((y_pred_list[0], y_pred_list[1], y_pred_list[2], y_pred_list[3], y_pred_list[4], y_pred_list[5]))
ids_test_sub = np.concatenate((ids_test_list[0], ids_test_list[1], ids_test_list[2], ids_test_list[3], ids_test_list[4], ids_test_list[5]))

OUTPUT_PATH = 'result.csv'
create_csv_submission(ids_test_sub, y_pred, OUTPUT_PATH)